# INTRODUÇÃO

A crescente competitividade no setor de telecomunicações tem impulsionado as empresas a compreenderem profundamente os padrões de rotatividade de clientes, conhecido como churn, para manter a lealdade e sustentar o crescimento. Neste contexto, nossa análise de dados visa responder a três perguntas fundamentais que orientarão estratégias eficazes de retenção de clientes.

1. Quais variáveis estão contribuindo para a rotatividade de clientes?
2. Quem são os clientes com maior probabilidade de abandono?
3. Que ações podem ser tomadas para impedi-los de sair?


Os dados utilizados foram extraídos de um conjunto de dados disponível no Kaggle. Para acessar o conjunto de dados, por favor, siga o link fornecido. https://www.kaggle.com/datasets/barun2104/telecom-churn

Definição das colunas da tabela:
 - Churn: Rotatividade do clientes.
 - AccountWeeks: Número de semanas que o cliente teve conta ativa.
 - ContractRenewal: Renovação de contrato com o cliente.
 - DataPlan: Clientes que possuem planos de dados.
 - DataUsage: Gigabytes de uso mensal de dados.
 - CustServCalls: Número de chamadas para atendimento ao cliente.
 - DayMins: Média de minutos diurnos por mês.
 - DayCalls: Número médio de chamadas diurnas.
 - MonthlyCharge: Faturamento médio mensal.
 - OverageFee: Maior taxa excedente nos últimos 12 meses.
 - RoamMins: Número médio de minutos de roaming.

# CARREGANDO BIBLIOTECAS

In [3]:
# Manipulação de dados
import numpy as np
import pandas as pd

# Análise exploratória
import matplotlib.pyplot as plt
import seaborn as sns

# Configuracoes
plt.style.use("ggplot")
pd.set_option("display.max_rows",20)
pd.set_option("display.max_columns",40)

# CARREGANDO DATASET

In [4]:
churn = pd.read_csv("data/telecom_churn.csv")
churn.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [6]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Churn            3333 non-null   int64  
 1   AccountWeeks     3333 non-null   int64  
 2   ContractRenewal  3333 non-null   int64  
 3   DataPlan         3333 non-null   int64  
 4   DataUsage        3333 non-null   float64
 5   CustServCalls    3333 non-null   int64  
 6   DayMins          3333 non-null   float64
 7   DayCalls         3333 non-null   int64  
 8   MonthlyCharge    3333 non-null   float64
 9   OverageFee       3333 non-null   float64
 10  RoamMins         3333 non-null   float64
dtypes: float64(5), int64(6)
memory usage: 286.6 KB


In [8]:
churn.nunique().sort_values()

Churn                 2
ContractRenewal       2
DataPlan              2
CustServCalls        10
DayCalls            119
RoamMins            162
DataUsage           174
AccountWeeks        212
MonthlyCharge       627
OverageFee         1024
DayMins            1667
dtype: int64

In [23]:
print(f"Nosso dataset possui {churn.shape[0]} linhas e {churn.shape[1]} colunas.")

Nosso dataset possui 3333 linhas e 11 colunas.


In [33]:
cliente_sem_plano = churn.query("DataPlan == 0")
cliente_com_plano = churn.query("DataPlan == 1")
cliente_sem_plano

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
5,0,118,0,0,0.00,0,223.4,98,57.0,11.03,6.3
7,0,147,0,0,0.00,0,157.0,79,36.0,5.16,7.1
...,...,...,...,...,...,...,...,...,...,...,...
3326,0,96,1,0,0.36,1,106.6,128,46.6,14.24,14.9
3327,0,79,1,0,0.00,2,134.7,98,40.0,9.49,11.8
3329,0,68,1,0,0.34,3,231.1,57,56.4,7.67,9.6
3330,0,28,1,0,0.00,2,180.8,109,56.0,14.44,14.1


In [38]:
print(f"A quatidade de clientes que possuem plano de dados é {cliente_com_plano.shape[0]} e que não possuem é {cliente_sem_plano.shape[0]}.")

A quatidade de clientes que possuem plano de dados é 922 e que não possuem é 2411.


A maior parte do dataset é de clientes que não possuem plano de dados. Esse dado tende a ser relevante para uma rotatividade de clientes, vale a pena investigar.

In [24]:
account_weeks_min = churn.AccountWeeks.min()
account_weeks_max = churn.AccountWeeks.max()

print(f"Nosso dataset possui clientes que vão de no minimo {account_weeks_min} a no máximo {account_weeks_max} semanas com a conta ativa.")

Nosso dataset possui clientes que vão de no minimo 1 a no máximo 243 semanas com a conta ativa.


In [39]:
churn["AccountWeeks"].describe()

count    3333.000000
mean      101.064806
std        39.822106
min         1.000000
25%        74.000000
50%       101.000000
75%       127.000000
max       243.000000
Name: AccountWeeks, dtype: float64